In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hebrew-questions/questions.xlsx


In [43]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Sat Jan 13 15:44:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [4]:
import os
questions = pd.read_excel('/kaggle/input/hebrew-questions/questions.xlsx',index_col=[0])
all_questions = questions['Q']

In [5]:
all_questions[0:4]

0    כמה אנשים היו בוועדה שניסחה את הטיוטה הרביעית?
1              באילו מילים התחילו כל סעיפי הטויוטה?
2         איזו תוכנית הוכרזה כחלק מההצעה של הטיוטה?
3       מתי לראשונה בהיסטוריה קמו המדינות הראשונות?
Name: Q, dtype: object

In [6]:
class QuestionDataset(Dataset):
    def __init__(self, questions, tokenizer):
        self.questions = questions
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question_text = self.questions[idx]
        encoding = self.tokenizer(question_text, return_tensors='pt', max_length=128, truncation=True,
                                  padding='max_length')
        return {"input_ids": encoding["input_ids"].squeeze(), "labels": encoding["input_ids"].squeeze()}


In [7]:
#https://huggingface.co/Norod78/distilgpt2-base-pretrained-he

model_name = "Norod78/distilgpt2-base-pretrained-he"

model_output_dir = "./fine_tuned_model"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

train_dataset = QuestionDataset(all_questions, tokenizer)

In [8]:
import wandb
wandb.init(mode='disabled')

In [9]:
training_args = TrainingArguments(
    output_dir=model_output_dir,
    overwrite_output_dir=True,
    report_to=None,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    num_train_epochs=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,1.567300
1000,0.361700
1500,0.360100
2000,0.353200
2500,0.347400
3000,0.352700
3500,0.352000
4000,0.343600
4500,0.342400
5000,0.346300


TrainOutput(global_step=6620, training_loss=0.4407640514777146, metrics={'train_runtime': 435.3592, 'train_samples_per_second': 60.819, 'train_steps_per_second': 15.206, 'total_flos': 864826920271872.0, 'train_loss': 0.4407640514777146, 'epoch': 1.0})

In [32]:
def generate_text(model, tokenizer, input_text, max_length=50,top_p=0.95,temperature = 0.95):
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=max_length, truncation=True).cuda()
    output_ids = model.cuda().generate(input_ids, max_length=max_length, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=top_p, temperature=temperature)
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text

user_input = 'ריבית'
generate_text(model, tokenizer, user_input, max_length=15,top_p=0.9,temperature=0.9)

'ריבית פריים גבוהה יותר של 3.5% בשנה?'

In [36]:
questions[questions['Q'].str.contains('ריבית')].head(5)

,Q
14272,מי יקבל את הריבית עם הרווחים?


In [34]:
user_input = 'למה מחשבון'
generate_text(model, tokenizer, user_input, max_length=15,top_p=0.9,temperature=0.9)

'למה מחשבון ה-Galaxy Note 8 של סמסונג?'

In [35]:
questions[questions['Q'].str.contains('מחשבון')].head(10)

,Q
15594,מי התפרסם בעקבות המצאת מחשבון הכיס האיקוני Sin...
15596,בכמה מכרו את המחשבון Sinclair Executive באותה ...
15597,כמה כסף הרוויחה החברה מהמחשבון של סינקלייר?
17266,מי המציא את המחשבון?
23884,מתי הוצג מחשבון מוטמע בשעון יד?
23886,"כמה עלה מחשבון מדעי בארה""ב ב 2017?"
23888,איזה מקור כוח שאינו סוללה קיים למחשבון?


In [41]:
user_input = 'סלט עגבניות'
generate_text(model, tokenizer, user_input, max_length=15,top_p=0.9,temperature=0.9)

"סלט עגבניות ופלפליית הפלנצ'ה של סבתא?"

In [42]:
questions[questions['Q'].str.contains('סלט עגבניות')].head(10)

,Q


In [27]:
model.save_pretrained("finetuned_model")
"""

Step	Training Loss
500	1.558400
1000	0.359200
1500	0.351400
2000	0.349200
2500	0.346000
3000	0.341200
TrainOutput(global_step=3310, training_loss=0.531809105541771, metrics={'train_runtime': 595.9882, 'train_samples_per_second': 44.427, 'train_steps_per_second': 5.554, 'total_flos': 864826920271872.0, 'train_loss': 0.531809105541771, 'epoch': 1.0})

"""